In [1]:
import argparse
import copy
import itertools
import warnings
import os

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from lightgbm import LGBMClassifier
from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.feature_selection import (
    SelectFromModel,
    SelectKBest,
    VarianceThreshold,
    f_classif,
)
from sklearn.metrics import (
    auc,
    matthews_corrcoef,
    precision_recall_fscore_support,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
)
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    StratifiedKFold,
    cross_val_predict,
    cross_val_score,
    train_test_split,
)
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, label_binarize, StandardScaler
from sklearn.svm import SVC

# from tabpfn import TabPFNClassifier
# from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNClassifier
from xgboost import XGBClassifier
from itertools import combinations

# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)



# OvO and OvR prediction function

In [10]:
def split_classes(X, y):
    return {
        (c1, c2): (X[(y == c1) | (y == c2)], y[(y == c1) | (y == c2)])
        for c1, c2 in itertools.combinations(np.unique(y), 2)
    }


def ovo_and_ova_multiclass_auc(X, y, base_clf, p_grid, random_state):
    results = {}
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    class_names = le.classes_

    # Stratified K-Folds
    inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

    ####################
    # One-vs-Rest Classification
    ####################
    print("Performing One vs Rest classification")

    #checking grid search enabled or not 
    if p_grid is not None:
        ovr_clf = GridSearchCV(
            estimator=OneVsRestClassifier(base_clf),
            param_grid=p_grid,
            cv=inner_cv,
            scoring="roc_auc_ovr",
        )
    else:
        ovr_clf = OneVsRestClassifier(base_clf)
   
    
    y_score = cross_val_predict(ovr_clf, X, y_encoded, cv=outer_cv, method="predict_proba") 
    y_pred = np.argmax(y_score, axis=1) 
    
    # Per-class metrics for OvR 
    per_class_precision = [] 
    per_class_recall = [] 
    per_class_f1 = [] 
    per_class_mcc = []

    for idx, cls in enumerate(class_names): 
        y_bin = (y_encoded == idx).astype(int) 
        cls_score = y_score[:, idx] 
        
        # Ensure minority class is positive 
        if np.sum(y_bin) > np.sum(1 - y_bin): 
            y_bin = 1 - y_bin 
            cls_score = 1 - cls_score 
            
        y_pred_bin = (y_pred == idx).astype(int) 
        precision, recall, f1, _ = precision_recall_fscore_support(y_bin, y_pred_bin, average="binary") 
        mcc = matthews_corrcoef(y_bin, y_pred_bin) 
        prec_curve, rec_curve, _ = precision_recall_curve(y_bin, cls_score) 
        pr_auc_val = auc(rec_curve, prec_curve) 
        roc_auc_val = roc_auc_score(y_bin, cls_score) 
        
        results[f"{cls} vs Rest - Precision"] = precision 
        results[f"{cls} vs Rest - Recall"] = recall 
        results[f"{cls} vs Rest - F1"] = f1 
        results[f"{cls} vs Rest - MCC"] = mcc 
        results[f"{cls} vs Rest - PR AUC"] = pr_auc_val 
        results[f"{cls} vs Rest - AUC"] = roc_auc_val 
        
        per_class_precision.append(precision) 
        per_class_recall.append(recall) 
        per_class_f1.append(f1) 
        per_class_mcc.append(mcc) 
    
    # Macro metrics OvR 

    macro_ovr_auc = np.mean([results[f"{cls} vs Rest - AUC"] for cls in class_names]) 
    macro_ovr_precision = np.mean(per_class_precision) 
    macro_ovr_recall = np.mean(per_class_recall) 
    macro_ovr_f1 = np.mean(per_class_f1) 
    macro_ovr_mcc = np.mean(per_class_mcc)
    macro_ovr_pr_auc = np.mean([results[f"{cls} vs Rest - PR AUC"] for cls in class_names])

    
    results["OvR Macro AUC"] = macro_ovr_auc
    results["OvR Macro Precision"] = macro_ovr_precision
    results["OvR Macro Recall"] = macro_ovr_recall
    results["OvR Macro F1"] = macro_ovr_f1 
    results["OvR Macro MCC"] =  macro_ovr_mcc
    results["OvR Macro PR AUC"] = macro_ovr_pr_auc 

    print(f"Macro AUC (OvR): {macro_ovr_auc:.4f}")
    print(f"Macro Precision (OvR): {macro_ovr_precision:.4f}")
    print(f"Macro Recall (OvR): {macro_ovr_recall:.4f}")
    print(f"Macro F1 (OvR): {macro_ovr_f1:.4f}")
    print(f"Macro MCC (OvR): {macro_ovr_mcc:.4f}")
    print(f"Macro PR AUC (OvR): {macro_ovr_pr_auc:.4f}")

    ####################
    # One-vs-One Classification
    ####################
    print("Performing One vs One classification")

    ovo_auc = {} 
    ovo_precision = {} 
    ovo_recall = {} 
    ovo_f1 = {} 
    ovo_mcc = {} 
    
    for c1, c2 in combinations(range(len(class_names)), 2): 
        mask = np.isin(y_encoded, [c1, c2]) 
        X_pair, y_pair = X[mask], y_encoded[mask] 

        # checking grid search enabled or not
        if p_grid is not None:
            ovo_clf = GridSearchCV(
                estimator=base_clf,
                param_grid={k.replace("estimator__", ""): v for k, v in p_grid.items()},
                cv=inner_cv,
                scoring="roc_auc"
            )
        else:
            ovo_clf = base_clf
            
        y_score_pair = cross_val_predict(ovo_clf, X_pair, y_pair, cv=outer_cv, method="predict_proba") 
        
        # Identify minority 
        
        vals, counts = np.unique(y_pair, return_counts=True) 
        minority = vals[np.argmin(counts)] 
        minority_idx = np.where([c1, c2] == minority)[0][0] 
        
        y_bin = (y_pair == minority).astype(int) 
        y_score_cls = y_score_pair[:, minority_idx] 
        
        # Ensure minority positive 
        
        if np.sum(y_bin) > np.sum(1 - y_bin): 
            y_bin = 1 - y_bin 
            y_score_cls = 1 - y_score_cls 
            
        y_pred_bin = (np.argmax(y_score_pair, axis=1) == minority_idx).astype(int)
                                                                             
        precision, recall, f1, _ = precision_recall_fscore_support(y_bin, y_pred_bin, average="binary") 
        mcc = matthews_corrcoef(y_bin, y_pred_bin) 
        prec_curve, rec_curve, _ = precision_recall_curve(y_bin, y_score_cls) 
        pr_auc_val = auc(rec_curve, prec_curve) 
        roc_auc_val = roc_auc_score(y_bin, y_score_cls)
        
        pair_name = f"{le.inverse_transform([c1])[0]} vs {le.inverse_transform([c2])[0]}" 
        
        results[f"{pair_name} - Precision"] = precision 
        results[f"{pair_name} - Recall"] = recall 
        results[f"{pair_name} - F1"] = f1 
        results[f"{pair_name} - MCC"] = mcc 
        results[f"{pair_name} - PR AUC"] = pr_auc_val 
        results[f"{pair_name} - AUC"] = roc_auc_val 
        
        ovo_auc[(c1, c2)] = roc_auc_val 
        ovo_precision[(c1, c2)] = precision 
        ovo_recall[(c1, c2)] = recall 
        ovo_f1[(c1, c2)] = f1 
        ovo_mcc[(c1, c2)] = mcc 
        
        
    # Macro metrics OvO 
    macro_ovo_auc = np.mean(list(ovo_auc.values()))
    macro_ovo_precision = np.mean(list(ovo_precision.values()))
    macro_ovo_recall = np.mean(list(ovo_recall.values())) 
    macro_ovo_f1 = np.mean(list(ovo_f1.values())) 
    macro_ovo_mcc = np.mean(list(ovo_mcc.values())) 
    macro_ovo_pr_auc = np.mean([results[k] for k in results if "vs" in k and "PR AUC" in k]) 

    results["OvO Macro AUC"] =  macro_ovo_auc
    results["OvO Macro Precision"] = macro_ovo_precision
    results["OvO Macro Recall"] = macro_ovo_recall
    results["OvO Macro F1"] = macro_ovo_f1
    results["OvO Macro MCC"] = macro_ovo_mcc
    results["OvO Macro PR AUC"] =  macro_ovo_pr_auc


    print(f"Macro AUC (OvO): {macro_ovo_auc:.4f}")
    print(f"Macro Precision (OvO): {macro_ovo_precision:.4f}")
    print(f"Macro Recall (OvO): {macro_ovo_recall:.4f}")
    print(f"Macro F1 (OvO): {macro_ovo_f1:.4f}")
    print(f"Macro MCC (OvO): {macro_ovo_mcc:.4f}")
    print(f"Macro PR AUC (OvO): {macro_ovo_pr_auc:.4f}")

    return results


def repeat_clf(n_seeds, ks, X, y, label, model, sampling_strategy, use_grid=False):

    print(ks)
    print(n_seeds)

    # Define sampling strategies
    sampling_strategies = {
        "No Sampling": None,
        "Random OverSampling": RandomOverSampler(random_state=42),
        "SMOTE": SMOTE(random_state=42),
        "Random UnderSampling": RandomUnderSampler(random_state=42),
        "NearMiss (v1)": NearMiss(version=1),
        "NearMiss (v2)": NearMiss(version=2),
        "NearMiss (v3)": NearMiss(version=3),
    }

    # If the selected strategy is not in the dictionary, use "No Sampling"
    sampler = sampling_strategies.get(sampling_strategy, None)

    seed_results = {}

    for seed in range(n_seeds):

        ks_results = {}
        for k in ks:

            print(f"CV for seed {seed} and {k} features")

            # Create a Random Forest Classifier
            rf = RandomForestClassifier(random_state=seed)

            # Create a SelectFromModel using the Random Forest Classifier
            selector = SelectFromModel(rf, max_features=k)

            if model == "rf":
                ml_model = rf
                ml_model_grid = {
                    "estimator__classification__n_estimators":[100, 300, 500],  # Number of trees in the forest
                    "estimator__classification__max_depth": [None, 10, 20, 30],  # tree depth
                    "estimator__classification__max_features": ["sqrt", "log2"],  # Feature selection strategy
                    "estimator__classification__criterion": ["entropy"],  # Split criterion
                    "estimator__classification__min_samples_leaf": [1, 2, 4],  # Minimum samples per leaf
                }
            elif model == "xgb":
                ml_model = XGBClassifier(
                    use_label_encoder=False, eval_metric="logloss", random_state=seed
                )
                ml_model_grid = {
                    "estimator__classification__n_estimators": [100, 300, 500], 
                    "estimator__classification__gamma": [0, 0.1, 0.3], # min loss reduction
                    "estimator__classification__max_depth": [3, 5, 7], 
                    "estimator__classification__learning_rate": [0.01, 0.05, 0.1], # step size
                }
            elif model == "etc":
                ml_model = ExtraTreesClassifier(random_state=seed)
                ml_model_grid = {
                    "estimator__classification__n_estimators": [100, 300, 500],
                    "estimator__classification__max_depth": [None, 10, 20],       # tree depth
                    "estimator__classification__max_features": ["sqrt", "log2"],  # features per split
                    "estimator__classification__min_samples_leaf": [1, 2, 4],     # min leaf samples
                    
                }
            elif model == "lgbm":
                ml_model = LGBMClassifier(random_state=seed, verbose=-1)
                ml_model_grid = {
                    "estimator__classification__n_estimators": [100, 300, 500],  
                    "estimator__classification__learning_rate": [0.01, 0.05, 0.1],
                    "estimator__classification__num_leaves": [31, 63, 127],      # leaves per tree
                            
                }

            # If there is a sampler, include it in the pipeline
            steps = []
            if sampler:
                steps.append(("sampling", sampler))
            steps.append(("feature_selection", selector))
            steps.append(("classification", ml_model))

            # Create a pipeline with feature selection, sampling, and classification
            pipeline = Pipeline(steps=steps)

            ###########################

            # Run the classification with the sampling strategy
            if use_grid:
                results = ovo_and_ova_multiclass_auc(
                    X, y, pipeline, ml_model_grid, random_state=seed
                )
            else:
                results = ovo_and_ova_multiclass_auc(
                    X, y, pipeline, None, random_state=seed
                )
                       

            print(results)

            ks_results[k] = {
                "results": results,
                "Label": label,
                "Model": model,
                "Sampling_Strategy": sampling_strategy,
            }


        seed_results[seed] = copy.copy(ks_results)

    return seed_results


def store_results(seed_results, output):

    # Flatten the nested dictionary into a DataFrame
    '''df = pd.DataFrame(
        {
            (outer_key, inner_key): values
            for outer_key, inner_dict in seed_results.items()
            for inner_key, values in inner_dict.items()
        }
    ).T

    # '''
    
    final_results = []
    metrics = ["AUC", "Precision", "Recall", "F1", "MCC", "PR"]
    
    for seed, ks_results in seed_results.items():
        for k, result_info in ks_results.items():
            result = result_info["results"]
            model = result_info["Model"]
            sampling_strategy = result_info["Sampling_Strategy"]
            label=result_info["Label"]
                        
            # Determine Class and Type
           
            groups = set()
            for key in result.keys():
                if "Macro" in key:
                    groups.add(("Macro", "OvR" if "OvR" in key else "OvO"))
                elif "vs Rest" in key:
                    groups.add((key.split(" vs Rest")[0], "OvR"))
                else:
                    groups.add((key.split(" - ")[0], "OvO"))

            # assign metric values according to class and type
            for class_name, type_name in groups:

                metric_values = {}
            
                for metric in metrics:
                    metric_key = None
            
                    for key in result.keys():
                        if class_name == "Macro":
                            if metric in key and "Macro" in key and type_name in key:
                                metric_key = key
                                break
                        elif type_name == "OvR":
                            if metric in key and f"{class_name} vs Rest" in key:
                                metric_key = key
                                break
                        else:  # OvO
                            if metric in key and "vs" in key and class_name in key:
                                metric_key = key
                                break
            
                    metric_values[metric] = result[metric_key] if metric_key else np.nan

                final_results.append({
                    "Seed": seed,
                    "Features (k)": k,
                    "Label": label,
                    "Model": model,
                    "Sampling_Strategy": sampling_strategy,
                    "Class": class_name,
                    "Type": type_name,
                    **metric_values
                })
                
    df = pd.DataFrame(final_results)

    '''#Set multi-level index names for clarity
    df.set_index(["Seed", "Features (k)", "Label", "Model", "Sampling_Strategy"], inplace=True)

    df.index.names = ["Seed", "Features (k)","Label","Model","Sampling_Strategy"]
    # Display the DataFrame
    df = df.reset_index()'''

    df.to_csv(output, mode='a', header=not os.path.exists(output), index=False)

    print(df)


def run_classification(X, y, ks, n_seeds,output, label,model, sampling_strategy,use_grid=False):

    # Ensure ks does not exceed the number of columns in X
    max_features = len(X.columns)
    ks = [k for k in ks if k <= max_features]
    if max_features not in ks:
        ks.append(max_features)

    seed_results = repeat_clf(n_seeds, ks, X, y, label,model, sampling_strategy, use_grid=use_grid)
    store_results(seed_results, output)



# Import omics and metadata data

In [3]:
# Step 1: Define folder
output_dir = "HP_analysis_10_2025/input/HP_multiomics/"

# Step 2: Initialize list to store omics datasets
raw_data = []

# Step 3: Loop through all TSV files (excluding metadata and Virulence-Genes)
for file in os.listdir(output_dir):
    if (
        file.endswith("_clean.tsv") 
        and file != "metadata_clean.tsv"
        and not file.startswith("Virulence-Genes")
    ):
        name = file.replace("_clean.tsv", "")
        df = pd.read_csv(os.path.join(output_dir, file), sep="\t", index_col=0)
        raw_data.append({"name": name, "df": df})

# Step 4: Load metadata separately
metadata_path = os.path.join(output_dir, "metadata_clean.tsv")
y_aligned = pd.read_csv(metadata_path, sep="\t", index_col=0)

# Step 5: Print summary
print(f"Loaded {len(raw_data)} omics datasets:")
for omic in raw_data:
    print(f" - {omic['name']}: {omic['df'].shape[0]} samples × {omic['df'].shape[1]} features")

print(f"Metadata loaded with shape: {y_aligned.shape}")
print("\nDataset loading complete. (Virulence-Genes excluded)")

Loaded 6 omics datasets:
 - SCFA: 134 samples × 27 features
 - Metabolites: 134 samples × 439 features
 - GCMS: 134 samples × 99 features
 - Lipids: 134 samples × 622 features
 - 16S: 134 samples × 6241 features
 - RNA: 134 samples × 60839 features
Metadata loaded with shape: (134, 31)

Dataset loading complete. (Virulence-Genes excluded)


# Apply TSS and merge data

In [4]:
# --- Total Sum Scaling function ---
def total_sum_scale(df):
    df = df.fillna(0)
    return df.div(df.sum(axis=1), axis=0)

# --- Prepare TSS datasets ---
tss_datasets = []
dataset_names = []

for omics_data in raw_data:
    df_tss = total_sum_scale(omics_data["df"])
    tss_datasets.append(df_tss)
    dataset_names.append(omics_data["name"])

# --- Combined dataset ---
df_combined = pd.concat(tss_datasets, axis=1)
tss_datasets.append(df_combined)
dataset_names.append("Combined")

# --- Encode target ---
y_target = y_aligned["Sample_Condition"]

#print(df_combined)


# Test combined dataset - on all 4 models (no grid search) 

In [ ]:
n_seeds = 2
ks = [10]
result_path="HP_analysis_10_2025/benchmark/results/diff_models_on_combined(no grid).csv"
# Check if the file exists and remove it once
if os.path.exists(result_path):
    print(f"File {result_path} exists. Deleting it to start fresh.")
    os.remove(result_path)
    
for model in ['rf', 'xgb', 'etc', 'lgbm']:
    print(f"model: {model} ")
    print("Combined dataset scores:")
    run_classification(df_combined, y_target, ks, n_seeds,result_path,"combined", model, None,use_grid=False)

    

 # Model Performance by Metric (Macro Scores)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the results
df = pd.read_csv("HP_analysis_10_2025/benchmark/results/diff_models_on_combined(no grid).csv")

# Use only Macro scores to compare models
metrics = ['AUC', 'Precision', 'Recall', 'F1', 'MCC', 'PR']
df_macro = df[df['Class'] == 'Macro']

# Melt the dataframe for easier plotting
df_melt = df_macro.melt(id_vars=['Model'], value_vars=metrics, var_name='Metric', value_name='Score')

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(data=df_melt, x='Metric', y='Score', hue='Model', ci=None, edgecolor='black', linewidth=0.5)
plt.title("Model Performance by Metric (Macro Scores)")
plt.ylabel("Score")
plt.ylim(0, 1)  
plt.xticks(rotation=30)
plt.legend(title='Model')
plt.tight_layout()
plt.show()

# 2 out of n Dataset combinations

In [7]:
import itertools
import pandas as pd

# Exclude the already combined dataset at the end
num_datasets = len(tss_datasets) - 1

all_combinations = []  # Stores the actual dataframes for each combination
all_combination_names = []  # Stores the names of datasets in each combination

# Generate all 2-dataset combinations
for combo_indices in itertools.combinations(range(num_datasets), 2):
    combination_df = pd.concat([tss_datasets[i] for i in combo_indices], axis=1)
    all_combinations.append(combination_df) # Store the dataframe separately

     # Store the names of the datasets in this combination
    all_combination_names.append([dataset_names[i] for i in combo_indices]) 
print("total combinations : ", len(all_combination_names))
print(all_combination_names)

total combinations :  15
[['SCFA', 'Metabolites'], ['SCFA', 'GCMS'], ['SCFA', 'Lipids'], ['SCFA', '16S'], ['SCFA', 'RNA'], ['Metabolites', 'GCMS'], ['Metabolites', 'Lipids'], ['Metabolites', '16S'], ['Metabolites', 'RNA'], ['GCMS', 'Lipids'], ['GCMS', '16S'], ['GCMS', 'RNA'], ['Lipids', '16S'], ['Lipids', 'RNA'], ['16S', 'RNA']]


# Evaluating the best model from above (RF) across different dataset combinations.

In [ ]:
n_seeds = 2
ks = [10]
result_path = "HP_analysis_10_2025/benchmark/results/combinations_of_2_datasets(no_grid).csv"

# Check if the file exists and remove it once
if os.path.exists(result_path):
    print(f"File {result_path} exists. Deleting it to start fresh.")
    os.remove(result_path)

# Loop through all dataset combinations
for combo_df, combo_names in zip(all_combinations, all_combination_names):
    combo_label = "+".join(combo_names)  # e.g., "metabolomics+transcriptomics"
    print(f"\nRunning rf model on combination: {combo_label}")
    run_classification(combo_df, y_target, ks, n_seeds, result_path, combo_label, "rf", None, use_grid=False)

# Plot Macro Scores Across 2-Dataset Combinations

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the results
df = pd.read_csv("HP_analysis_10_2025/benchmark/results/combinations_of_2_datasets(no_grid).csv")

# Keep only Macro class scores
df_macro = df[df['Class'] == 'Macro']

# List of metrics to plot (adjust based on your CSV)
metrics = ['AUC', 'Precision', 'Recall', 'F1', 'MCC', 'PR']

#  Ranked Barplots per metric ---
for metric in metrics:
    plt.figure(figsize=(12,6))
    df_sorted = df_macro.sort_values(metric, ascending=False)
    sns.barplot(data=df_sorted, x='Label', y=metric, palette='viridis', ci=None)
    plt.title(f"{metric} Ranked Across Dataset Combinations (RF Model)")
    plt.ylabel(f" Macro {metric} ")
    plt.ylim(0, 1)
    plt.xticks(rotation=45, ha='right')
    plt.xlabel("Dataset Combination")
    plt.tight_layout()
    plt.show()

    
   

# Plot all Metric scores for top 5 dataset combinations 

In [ ]:
# Load the results
df = pd.read_csv("HP_analysis_10_2025/benchmark/results/combinations_of_2_datasets(no_grid).csv")

# Keep only Macro scores
df_macro = df[df['Class'] == 'Macro']

# List of metrics
metrics = ['AUC', 'Precision', 'Recall', 'F1', 'MCC', 'PR']

# Compute overall mean per label
label_means = df_macro.groupby('Label')[metrics].mean().reset_index()

# Select top 5 dataset combinations
label_means['Overall'] = label_means[metrics].mean(axis=1)
top5_df = label_means.sort_values('Overall', ascending=False).head(5)


# Melt the dataframe for seaborn plotting
df_long = top5_df.melt(id_vars='Label', value_vars=metrics, var_name='Metric', value_name='Score')


# Plot grouped barplot for top 5 combinations
plt.figure(figsize=(10,6))
sns.barplot(data=df_long, x='Label', y='Score', hue='Metric', ci=None)
plt.xticks(rotation=30, ha='right')
plt.ylim(0, 1)
plt.title("Top 5 Dataset Combinations Across All Metrics (RF Model)")
plt.xlabel("Dataset Combination")
plt.ylabel("Macro Scores")
plt.legend(title='Metric')
plt.tight_layout()
plt.show()



   

# Hyperparameter Tuning for Random Forest using Grid Search

In [ ]:
n_seeds = 2
ks = [10]
result_path="HP_analysis_10_2025/benchmark/results/Hyperparameter_tuning_rf(grid_search).csv"

# Check if the file exists and remove it once
if os.path.exists(result_path):
    print(f"File {result_path} exists. Deleting it to start fresh.")
    os.remove(result_path)
    

run_classification(df_combined, y_target, ks, n_seeds,result_path,"combined", "rf", None,use_grid=True)

    

# graph 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the results
df = pd.read_csv("HP_analysis_10_2025/benchmark/results/Hyperparameter_tuning_rf(grid_search).csv")

# Keep only Macro scores and RF model
df_macro = df[(df['Class'] == 'Macro') & (df['Model'] == 'rf')]

# List of metrics
metrics = ['AUC', 'Precision', 'Recall', 'F1', 'MCC', 'PR']

# Melt the dataframe for seaborn plotting
df_long = df_macro.melt(id_vars=['Features (k)'], value_vars=metrics, var_name='Metric', value_name='Score')

# Plot grouped barplot with feature count as x-axis
plt.figure(figsize=(12,6))
sns.barplot(data=df_long, x='Features (k)', y='Score', hue='Metric', ci=None)
plt.ylim(0, 1)
plt.title("Random Forest Macro Scores Across Different Feature Counts")
plt.xlabel("Number of Features (k)")
plt.ylabel("Macro Scores")
plt.legend(title='Metric', bbox_to_anchor=(1.05,1), loc='upper left')
plt.tight_layout()
plt.show()


# Testing 

In [9]:
n_seeds = 2
ks = [10]
result_path="HP_analysis_10_2025/benchmark/results/testing.csv"
# Check if the file exists and remove it once
if os.path.exists(result_path):
    print(f"File {result_path} exists. Deleting it to start fresh.")
    os.remove(result_path)
    

run_classification(df_combined, y_target, ks, n_seeds,result_path,"combined", "rf", None,use_grid=False)

    

File HP_analysis_10_2025/benchmark/results/testing.csv exists. Deleting it to start fresh.
[10, 68267]
2
CV for seed 0 and 10 features
Performing One vs Rest classification
Macro AUC (OvR): 0.6698
Macro Precision (OvR): 0.4195
Macro Recall (OvR): 0.3926
Macro F1 (OvR): 0.3289
Macro MCC (OvR): 0.0719
Macro PR AUC (OvR): 0.4313
Performing One vs One classification
Macro AUC (OvO): 0.6592
Macro Precision (OvO): 0.4858
Macro Recall (OvO): 0.2667
Macro F1 (OvO): 0.3416
Macro MCC (OvO): 0.2072
Macro PR AUC (OvO): 0.4500
{'Negative control vs Rest - Precision': 0.29411764705882354, 'Negative control vs Rest - Recall': 0.7777777777777778, 'Negative control vs Rest - F1': 0.4268292682926829, 'Negative control vs Rest - MCC': np.float64(-0.2487145971214342), 'Negative control vs Rest - PR AUC': np.float64(0.5004385697013709), 'Negative control vs Rest - AUC': np.float64(0.5749063670411986), 'Patient vs Rest - Precision': 0.7142857142857143, 'Patient vs Rest - Recall': 0.3333333333333333, 'Patien

/home/renu/miniconda3/envs/planemo-env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Macro AUC (OvO): 0.7391
Macro Precision (OvO): 0.4444
Macro Recall (OvO): 0.1333
Macro F1 (OvO): 0.1979
Macro MCC (OvO): 0.1859
Macro PR AUC (OvO): 0.5265
{'Negative control vs Rest - Precision': 0.3153846153846154, 'Negative control vs Rest - Recall': 0.9111111111111111, 'Negative control vs Rest - F1': 0.4685714285714286, 'Negative control vs Rest - MCC': np.float64(-0.24668745581139884), 'Negative control vs Rest - PR AUC': np.float64(0.6658272755275458), 'Negative control vs Rest - AUC': np.float64(0.7204744069912609), 'Patient vs Rest - Precision': 0.5, 'Patient vs Rest - Recall': 0.06666666666666667, 'Patient vs Rest - F1': 0.11764705882352941, 'Patient vs Rest - MCC': np.float64(0.15149987190590394), 'Patient vs Rest - PR AUC': np.float64(0.6076966297607611), 'Patient vs Rest - AUC': np.float64(0.7983193277310925), 'Positive control vs Rest - Precision': 0.5, 'Positive control vs Rest - Recall': 0.03333333333333333, 'Positive control vs Rest - F1': 0.0625, 'Positive control vs R